In [30]:
import pandas as pd 
import os
import csv

In [31]:
# pulling in clean data 

Table2a_filepath = os.path.join("Clean Data\Table 2a.csv")
Table2b_filepath = os.path.join("Clean Data\Table 2b.csv")
Table3_filepath = os.path.join("Clean Data\Table 3.csv")

US_renewable = pd.read_csv(Table2a_filepath)
US_oil = pd.read_csv(Table2b_filepath)
USD_data = pd.read_csv(Table3_filepath)


In [32]:
#group by source types 

US_oil.groupby(["Description"]).sum()


,Value
Description,
"Electricity Net Generation From Coal, All Sectors",2.393215e+07
"Electricity Net Generation From Conventional Hydroelectric Power, All Sectors",4.102541e+06
"Electricity Net Generation From Natural Gas, All Sectors",1.681585e+07
"Electricity Net Generation From Other Gases, All Sectors",1.885745e+05
"Electricity Net Generation From Petroleum, All Sectors",6.027747e+05


In [14]:
#switching index to dates

US_renewable = US_renewable.set_index(US_renewable["Date"], drop=True)
US_oil = US_oil.set_index(US_oil["Date"], drop=True)
USD_data = USD_data.set_index(USD_data["Date"], drop=True)

US_oil = US_oil.drop(columns=["Date", "Description"])
US_renewable = US_renewable.drop(columns=["Date", "Description"])
USD_data = USD_data.drop(columns=["Date"])


In [16]:
#renaming columns 



,Value
Date,
2005-01-01,177013.907
2005-02-01,155818.014
2005-03-01,163612.700
2005-04-01,143083.239
2005-05-01,153957.962
